<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN09/9.1-Text_als_sequenzielle_Daten_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

This notebook contains the first code sample found in Chapter 6, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff).

In diesem Kapitel werden wir Deep-Learning-Modelle erkunden, die Texte (in Form von Wortsequenzen oder Zeichenfolgen), Zeitreihen und sequenzielle Daten im Allgemeinen verarbeiten können. Die beiden grundlegenden Deep-Learning-Algorithmen
für die sequenzielle Verarbeitung von Daten sind **rekurrente neuronale Netze (RNNs) und 1-D-CNNs**, die eindimensionale Variante der CNNs, die wir in den vorangegangenen Kapiteln erörtert haben. In diesem Kapitel werden wir beide
Ansätze betrachten. 

Die Anwendungsmöglichkeiten dieser Algorithmen sind vielfältig:

- **Klassifizierung von Dokumenten** und Zeitreihen, wie beispielsweise die Erkennung des Themas eines Artikels oder des Autors eines Buchs
- Vergleich von Zeitreihen, beispielsweise die Abschätzung, wie eng verwandt zwei Dokumente oder zwei Börsenkürzel sind
- **Übersetzung:** Erlernen der Umwandlung von einer Sequenz in eine andere, wie z.B. die Übertragung eines englischen Satzes ins Französische
- **Stimmungsanalyse** (sentiment analysis), wie die Klassifizierung von Tweets oder Filmbewertungen als positiv oder negativ
- **Zeitreihenvorhersage**, wie etwa die Prognose des zukünftigen Wetters an einem bestimmten Ort anhand der vorangegangenen Wetterdaten

# 9. Text als sequenzielle Daten und Anwendungen von NLP
### (NLP=Natural Language Processing)

Text ist die wohl verbreitetste Form **sequenzieller Daten**. Man kann Text als eine Sequenz von Zeichen oder Wörtern auffassen, aber üblicherweise betrachtet man die Wörter. 

Die in den folgenden Abschnitten vorgestellten Deep-Learning-Modelle zur Verarbeitung von Texten können ein elementares Verständnis natürlicher Sprache erreichen, das für folgende Anwendungen ausreicht:
- die **Klassifizierung** von Dokumenten,
- die **Stimmungsanalyse** (sentiment analysis), 
- die **Erkennung von Autoren** oder sogar das 
- **Beantworten von Fragen** (in eingeschränktem Kontext, chatbots).

Sie sollten sich bei der Lektüre dieses Kapitels natürlich darüber im Klaren sein, dass **keins dieser Deep-Learning-Modelle einen Text wirklich so wie ein Mensch versteht**. 
- Die Modelle können jedoch die statistische Struktur der Schriftsprache erkennen, und das genügt, um viele einfache textbezogene Aufgaben zu lösen. 
- Bei der Verarbeitung natürlicher Sprache durch Deep Learning wird die Mustererkennung auf Wörter, Sätze und Textpassagen angewendet, ähnlich wie beim maschinellen Sehen die Mustererkennung auf Pixel angewandt wird.

### Vektorisierung

Ebenso wie alle anderen NNs nehmen Deep-Learning-Modelle keinen reinen Text als Eingabe entgegen, sondern numerische Tensoren. 
- Die **Umwandlung von Text in numerische Tensoren wird als Vektorisierung bezeichnet** und kann auf verschiedene Weise durchgeführt werden:

- Teilen Sie den Text in *Wörter* auf und wandeln Sie die einzelnen Wörter in Vektoren um.
- Teilen Sie den Text in *Zeichen* auf und wandeln Sie die einzelnen Zeichen in Vektoren um.
- Extrahieren Sie *N-Gramme* aus Wörtern oder Zeichen und wandeln Sie die einzelnen N-Gramme in Vektoren um. 

Bei der Verarbeitung natürlicher Sprache werden zusammenhängende Textfragmente – Wörter, Buchstaben oder Symbole – als **N-Gramme** bezeichnet.

<img src="Bilder/Token_Vektorisierung.png" width="340" height="340" align="center"/>
Aus einem Text werden Tokens, und aus den Tokens werden Vektoren.

### Tokens

- Die verschiedenen Bestandteile, in die Sie einen Text zerlegen können (**Wörter, Zeichen oder N-Gramme**), bezeichnet man zusammengenommen als **Tokens** und die Aufteilung von Text in solche Tokens als **Tokenisierung**.
- Bei jeder Vektorisierung von Text kommt irgendeine Form der Tokenisierung zum Einsatz, die den erzeugten Tokens einen numerischen Vektor zuordnet.
- Diese Vektoren werden zu **Sequenztensoren** gebündelt und in DNNs eingespeist. 

Es gibt mehrere Möglichkeiten, einem Token einen Vektor zuzuordnen. In diesem Abschnitt werden
wir die beiden wichtigsten behandeln: 
1.  die *One-hot-Codierung* von Tokens und
2. die *Tokeneinbettung*, die typischerweise ausschliesslich für Wörter verwendet wird, was man dann als Worteinbettung (word embedding) bezeichnet. 


### N-Gramme und das Bag-of-words-Modell

N-Gramme von Wörtern sind Gruppen von N (oder weniger) aufeinanderfolgenden Wörtern, die Sie einem Satz entnehmen können. Dasselbe Konzept ist auch auf Zeichen statt Wörter anwendbar.

Betrachten Sie als einfaches Beispiel den Satz »The cat sat on the mat.«, der in Bigramme (N-Gramme der Größe 2) zerlegt werden kann:

In [ ]:
{
    "The",
    "The cat",
    "cat",
    "cat sat",
    "sat",
    "sat on",
    "on",
    "on the",
    "the",
    "the mat",
    "mat",
}


Der Satz könnte auch in die folgende Menge von Trigrammen zerlegt werden:

In [ ]:
{
    "The",
    "The cat",
    "cat",
    "cat sat",
    "The cat sat",
    "sat",
    "sat on",
    "on",
    "cat sat on",
    "on the",
    "the",
    "sat on the",
    "the mat",
    "mat",
    "on the mat",
}


So eine Menge wird als **Bag-of-words** (»Beutel-voller-Wörter«, in diesem Fall Bag of-bigrams bzw. Bag-of-trigrams) bezeichnet. Der Begriff »Bag« weist darauf hin, dass Sie es nicht mit einer Liste oder einer Sequenz, sondern mit einer Menge
von Tokens zu tun haben, denn die Tokens besitzen keine bestimmte Reihenfolge.
Diese Methoden der Tokenisierung heissen **Bag-of-words-Modelle**.

# One-hot-Codierung von Wörtern und Zeichen

Die **One-hot-Codierung** ist die gebräuchlichste und einfachste Methode, ein Token in einen Vektor umzuwandeln. Wir haben die One-hot-Codierung zuvor schon bei den IMDb- und Reuters-Beispielen in Aktion gesehen (die Codierung von Wörtern in diesen beiden Beispielen). 

- Bei der One-hot-Codierung wird jedem Wort ein eindeutiger Integerindex zugeordnet. 
- Dieser Index `i` wird anschliessend in einen binären Vektor der Größe `N` umgewandelt (`N` bezeichnet die Grösse des Vokabulars (dictionaries), also die Anzahl unterschiedlicher Wörter). 
- Der Vektor enthält bis auf eine Ausnahme nur Nullen, das i-te Element ist 1. 
- Die One-hot-Codierung ist natürlich auch auf Zeichen anwendbar. 

Um zu verdeutlichen, wie die One-hot-Codierung funktioniert und wie sie implementiert wird, finden Sie in den folgenden Listings zwei Beispiele. Das erste codiert Wörter und das zweite Zeichen.


In [ ]:
import numpy as np

# This is our initial data; one entry per "sample"
# (in this toy example, a "sample" is just a sentence, but
# it could be an entire document).
samples = ["the cat sat on the mat.", "the dog ate my homework."]

# First, build an index of all tokens in the data.
token_index = {}
i = 0
for sample in samples:
    # We simply tokenize the samples via the `split` method.
    # in real life, we would also strip punctuation and special characters
    # from the samples.
    for word in sample.split():
        print("(%i %s)" % (i + 1, word))
        if word not in token_index:
            i += 1
            # Assign a unique index to each unique word (dictionary)
            token_index[word] = len(token_index) + 1
            # Note that we don't attribute index 0 to anything.


In [ ]:
token_index


In [ ]:
# Next, we vectorize our samples.
# We will only consider the first `max_length` words in each sample.
max_length = 10

# This is where we store our results:
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.0


In [ ]:
results.shape


In [ ]:
list(enumerate(sample.split()))[:max_length]


In [ ]:
token_index.get("dog")


In [ ]:
samples[1]


In [ ]:
list(enumerate(samples[0].split()))[:max_length]


In [ ]:
list(enumerate(samples[1].split()))[:max_length]


In [ ]:
results[1, :, :]


#### Character level one-hot encoding (toy example)

In [ ]:
import string

samples = ["The cat sat on the mat.", "The dog ate my homework."]
characters = string.printable  # All printable ASCII characters.
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.0


In [ ]:
results.shape


In [ ]:
results[1, :, :]


### Tokenizer

Beachten Sie, dass **Hilfsfunktionen** für die One-hot-Codierung von Wörtern
und Zeichen integriert sind. 
- Sie sollten diese Hilfsfunktionen auch verwenden, denn sie bieten eine Reihe wichtiger Features, wie das Entfernen von Sonderzeichen oder dass nur die N am häufigsten in der Datenmenge vorkommenden Wörter berücksichtigt werden (eine häufig vorgenommene Beschränkung, die sehr grosse Eingabevektoren verhindern soll).

Der Download von `punkt_tab` muss im lokalen Verzeichnis der virtuellen Umgebung geschehen:
```python
import nltk
nltk.download('punkt_tab')
```

In [ ]:
import nltk

nltk.download("punkt_tab")

In [ ]:
import nltk
from collections import defaultdict
from pprint import pprint

nltk.download("punkt")  # Nur beim ersten Mal nötig

samples = ["The cat sat on the mat.", "The dog ate my homework."]

# 1. Tokenisieren der Sätze in Wörter (word_tokenize macht auch . und , einzeln)
tokenized_samples = [nltk.word_tokenize(sent.lower()) for sent in samples]

# 2. Vokabular erstellen
word_index = {}
index = 1  # Startindex bei 1
for sentence in tokenized_samples:
    for word in sentence:
        if word not in word_index:
            word_index[word] = index
            index += 1

# 3. Texte in Sequenzen von Integern umwandeln (wie texts_to_sequences)
sequences = [[word_index[word] for word in sentence] for sentence in tokenized_samples]

print(f"Found {len(word_index)} unique tokens.")
pprint(word_index)
print("Sequences:", sequences)


### One-hot-Hashing-Trick

Es gibt eine Variante der One-hot-Codierung, den sogenannten *One-hot-Hashing-Trick*, den Sie verwenden können, wenn die Anzahl unterschiedlicher Tokens in Ihrem Vokabular zu gross ist, um sie explizit zu handhaben. 
- Anstatt jedem Wort ausdrücklich einen Index zuzuweisen und eine Referenz auf diese Indizes in einem Dictionary zu speichern, können Sie mit einer *Hashfunktion Vektoren fester Grösse* erzeugen. 
- Dazu wird typischerweise eine äusserst **leichtgewichtige (d.h. einfach berechenbare) Hashfunktion** verwendet. Der Hauptvorteil dieser Methode besteht darin, dass die Notwendigkeit entfällt, einen expliziten Wortindex verwalten zu müssen. 
- Das spart Speicherplatz und gestattet es, die **Daten in Echtzeit zu codieren**, denn Sie können die Tokenvektoren schon erzeugen, bevor sämtliche Daten verfügbar sind.



Dieser Ansatz hat allerdings den Nachteil, dass er für **Hashkollisionen** anfällig ist:
- Zwei verschiedenen Wörtern könnte derselbe Hashwert zugewiesen werden. 
- Ein Machine-Learning-Modell wäre dann nicht mehr in der Lage, diese beiden Wörter zu unterscheiden. 
- Die Wahrscheinlichkeit für Hashkollisionen sinkt, wenn die Dimensionalität des Hashraums beträchtlich grösser ist als die Gesamtzahl der eindeutigen Tokens, für die Hashwerte erzeugt werden.

Hier als Beispiel ein Word-level one-hot encoding mit dem hashing trick (toy example):

In [ ]:
import torch

samples = ["The cat sat on the mat.", "The dog ate my homework."]
dimensionality = 1000  # Größe der Hash-Vektoren
max_length = 10  # Maximale Anzahl Wörter pro Sample

# Initialisiere den Tensor mit Nullen
results = torch.zeros((len(samples), max_length, dimensionality))

# Iteriere über die Samples
for i, sample in enumerate(samples):
    words = sample.replace(".", "").split()  # Entferne Punkt am Ende
    for j, word in list(enumerate(words))[:max_length]:
        # Berechne Hash-Code und bring ihn in Bereich [0, dimensionality)
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.0
        print(f"{word} \thash code: {index}")


In [ ]:
results.shape


In [ ]:
abs(hash("The")) % dimensionality
